In [1]:
import pandas as pd
import numpy as np
from scipy.special import softmax
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from transformers import Trainer
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer, DataCollatorWithPadding, BertForSequenceClassification

from datasets import load_metric
from datasets import load_dataset
from datasets import load_from_disk

import helper_data, helper_model
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Howard\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Howard\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import torch
torch.cuda.empty_cache()

In [3]:
import os
print(os.getcwd())

d:\OneDrive\Cornell\Fall 2023\Analyzing-the-Correlation-Between-Retail-Traders--Sentiments-and-Equity-Market-Movements\Sentiment_Analysis


In [4]:
# !pip install datasets

In [5]:
# raw_datasets = load_from_disk("data")
raw_datasets = load_from_disk("data/finetune_data/")
checkpoint = "bert-base-uncased"
checkpoint = "bert-base-cased"
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["text"], padding=True, truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # pad all the examples to the length of the longest element when we batch elements together — dynamic padding.

Map:   0%|          | 0/1081 [00:00<?, ? examples/s]

In [6]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4320
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1081
    })
    test: Dataset({
        features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5656
    })
})

In [7]:
raw_datasets['train']

Dataset({
    features: ['text', 'label'],
    num_rows: 4320
})

In [8]:
tokenized_datasets['train']['text']

['$AAPL shows signs of recovery after the slump. ????',
 "$JPM's stock faces downward pressure as the company navigates regulatory challenges in the banking industry.",
 '$GOOG just reported robust earnings. Stock is soaring. ????',
 '$ROKU streams in the wrong direction with intensifying competition in the streaming market.',
 "$GME's stock soars to new heights, triggering concerns of a speculative bubble reminiscent of the past.",
 'Travis Perkins Hikes Dividend 20% As Profit And Revenue Rise',
 'The stock rose for a second day on Wednesday bringing its two-day rise to GBX12 .0 or 2.0 % .',
 "$DIS's stock experiences volatility as the company deals with theme park closures and delays in movie releases.",
 '@Intuitiv $CLNE gasoline prices spike in California. http://stks.co/t1bOi',
 "$DIS's stock takes a hit as the company deals with theme park closures and delays in movie releases.",
 'Both operating profit and net sales for the six-month period increased , respectively from EUR7 .5 

In [9]:
metric = load_metric("glue", "mrpc")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

<ipython-input-9-093e3287862a>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue", "mrpc")


In [10]:
# !pip install transformers[torch]
# !pip install accelerate -U

In [11]:
# !pip install accelerate==0.24.0

In [12]:
import accelerate
print(accelerate.__version__)

0.24.1


In [30]:
training_args = TrainingArguments(
    output_dir="test-trainer",
    evaluation_strategy="epoch",
    num_train_epochs=50,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=256,
    # During the first 500 training steps, the learning rate gradually increases from 0 (or a small base rate) to the specified learning rate.
    # This gradual increase helps in stabilizing the training process and often leads to better performance, as it prevents the model from making too large updates too quickly.
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='logs',
    logging_steps=10,  # how frequently the training progress is logged
    save_strategy="epoch",  # Set save strategy to match evaluation strategy
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none",  # disable wandb
    fp16=True,  # Enable mixed precision training
)
# model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
model = BertForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to(device)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [31]:
trainer.train()

c:\Users\Howard\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


0%|          | 0/13500 [00:00<?, ?it/s]

{'loss': 0.6841, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.04}
{'loss': 0.6867, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.07}
{'loss': 0.6759, 'learning_rate': 3e-06, 'epoch': 0.11}
{'loss': 0.6667, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.15}
{'loss': 0.6582, 'learning_rate': 5e-06, 'epoch': 0.19}
{'loss': 0.6204, 'learning_rate': 6e-06, 'epoch': 0.22}
{'loss': 0.607, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.26}
{'loss': 0.5432, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.3}
{'loss': 0.5105, 'learning_rate': 9e-06, 'epoch': 0.33}
{'loss': 0.4587, 'learning_rate': 1e-05, 'epoch': 0.37}
{'loss': 0.399, 'learning_rate': 1.1000000000000001e-05, 'epoch': 0.41}
{'loss': 0.3849, 'learning_rate': 1.2e-05, 'epoch': 0.44}
{'loss': 0.3854, 'learning_rate': 1.3000000000000001e-05, 'epoch': 0.48}
{'loss': 0.3176, 'learning_rate': 1.3900000000000002e-05, 'epoch': 0.52}
{'loss': 0.3166, 'learning_rate': 1.49e-05, 'epoch': 0.56}
{'loss': 0.273, 'learning

0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 0.2005515843629837, 'eval_accuracy': 0.9324699352451434, 'eval_f1': 0.9318394024276377, 'eval_runtime': 4.5346, 'eval_samples_per_second': 238.39, 'eval_steps_per_second': 1.103, 'epoch': 1.0}
{'loss': 0.1357, 'learning_rate': 2.7800000000000005e-05, 'epoch': 1.04}
{'loss': 0.088, 'learning_rate': 2.88e-05, 'epoch': 1.07}
{'loss': 0.1889, 'learning_rate': 2.98e-05, 'epoch': 1.11}
{'loss': 0.1042, 'learning_rate': 3.08e-05, 'epoch': 1.15}
{'loss': 0.0819, 'learning_rate': 3.18e-05, 'epoch': 1.19}
{'loss': 0.3905, 'learning_rate': 3.2800000000000004e-05, 'epoch': 1.22}
{'loss': 0.1159, 'learning_rate': 3.38e-05, 'epoch': 1.26}
{'loss': 0.1018, 'learning_rate': 3.48e-05, 'epoch': 1.3}
{'loss': 0.1418, 'learning_rate': 3.58e-05, 'epoch': 1.33}
{'loss': 0.145, 'learning_rate': 3.68e-05, 'epoch': 1.37}
{'loss': 0.199, 'learning_rate': 3.7800000000000004e-05, 'epoch': 1.41}
{'loss': 0.1046, 'learning_rate': 3.8700000000000006e-05, 'epoch': 1.44}
{'loss': 0.2, 'learning_rate': 3.

...

0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 0.5183690190315247, 'eval_accuracy': 0.9518963922294172, 'eval_f1': 0.9519408502772643, 'eval_runtime': 4.2257, 'eval_samples_per_second': 255.816, 'eval_steps_per_second': 1.183, 'epoch': 49.0}
{'loss': 0.0, 'learning_rate': 1.0269230769230772e-06, 'epoch': 49.04}
{'loss': 0.0, 'learning_rate': 9.884615384615385e-07, 'epoch': 49.07}
{'loss': 0.0, 'learning_rate': 9.5e-07, 'epoch': 49.11}
{'loss': 0.0, 'learning_rate': 9.115384615384615e-07, 'epoch': 49.15}
{'loss': 0.0, 'learning_rate': 8.730769230769232e-07, 'epoch': 49.19}
{'loss': 0.0, 'learning_rate': 8.346153846153847e-07, 'epoch': 49.22}
{'loss': 0.0, 'learning_rate': 7.961538461538461e-07, 'epoch': 49.26}
{'loss': 0.0, 'learning_rate': 7.576923076923077e-07, 'epoch': 49.3}
{'loss': 0.0, 'learning_rate': 7.192307692307693e-07, 'epoch': 49.33}
{'loss': 0.0, 'learning_rate': 6.807692307692308e-07, 'epoch': 49.37}
{'loss': 0.0, 'learning_rate': 6.423076923076923e-07, 'epoch': 49.41}
{'loss': 0.0, 'learning_rate': 6.03

0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 0.5192036628723145, 'eval_accuracy': 0.9518963922294172, 'eval_f1': 0.9519408502772643, 'eval_runtime': 4.4437, 'eval_samples_per_second': 243.268, 'eval_steps_per_second': 1.125, 'epoch': 50.0}
{'train_runtime': 5199.1178, 'train_samples_per_second': 41.546, 'train_steps_per_second': 2.597, 'train_loss': 0.020438045919493393, 'epoch': 50.0}


TrainOutput(global_step=13500, training_loss=0.020438045919493393, metrics={'train_runtime': 5199.1178, 'train_samples_per_second': 41.546, 'train_steps_per_second': 2.597, 'train_loss': 0.020438045919493393, 'epoch': 50.0})

In [38]:
predictions = trainer.predict(tokenized_datasets["test"])
print(predictions.predictions.shape, predictions.label_ids.shape)

0%|          | 0/23 [00:00<?, ?it/s]

(5656, 2) (5656,)


In [39]:
predictions

PredictionOutput(predictions=array([[-4.12 ,  3.912],
       [-4.707,  4.586],
       [ 5.75 , -6.008],
       ...,
       [ 5.688, -6.133],
       [-4.766,  4.64 ],
       [-4.785,  4.66 ]], dtype=float16), label_ids=array([1, 1, 1, ..., 1, 1, 1], dtype=int64), metrics={'test_loss': 1.9729912281036377, 'test_accuracy': 0.7625530410183875, 'test_f1': 0.8185380354006215, 'test_runtime': 11.1195, 'test_samples_per_second': 508.658, 'test_steps_per_second': 2.068})

In [40]:
# Apply softmax to convert logits to probabilities
probabilities = softmax(predictions.predictions, axis=1)

# Get the predicted class labels
predicted_labels = np.argmax(probabilities, axis=1)

print("Probabilities:\n", probabilities)
print("Predicted Labels:\n", predicted_labels)

Probabilities:
 [[3.252e-04 1.000e+00]
 [9.173e-05 1.000e+00]
 [1.000e+00 7.808e-06]
 ...
 [1.000e+00 7.331e-06]
 [8.219e-05 1.000e+00]
 [7.904e-05 1.000e+00]]
Predicted Labels:
 [1 1 0 ... 0 1 1]


In [41]:
true_labels = tokenized_datasets["test"]["label"]

accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels, average='weighted')
f1 = f1_score(true_labels, predicted_labels, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.7625530410183875
Precision: 0.8038747346072187
Recall: 0.7625530410183875
F1 Score: 0.7606205495221969


In [36]:
helper_model.print_wrong_classifications(predicted_labels, true_labels, raw_datasets['test'])

Predicted Label: 0, True Label: 1
Original Text: user I'd be afraid to short AMZN - they are looking like a near-monopoly in eBooks and infrastructure-as-a-service

Predicted Label: 1, True Label: 0
Original Text: AAP - user if so then the current downtrend will break. Otherwise just a short-term correction in med-term downtrend.

Predicted Label: 0, True Label: 1
Original Text: It really worries me how everyone expects the market to rally now,usually exact opposite happens every time we shall see soon bac spx jpm

Predicted Label: 1, True Label: 0
Original Text: Won't believe AAP uptrend is back until it crosses above MA(50)

Predicted Label: 0, True Label: 1
Original Text: SWY - 30% of float short and breaking out - ouch

Predicted Label: 0, True Label: 1
Original Text: CHDN - ong   49.02. Trailing Stop  56.66 from 6 prior Stops of 54.17, 49.88, 49.82, 45.47, 43.02 & 41.92 -  

Predicted Label: 1, True Label: 0
Original Text: user: been adding VXY long off the bottom today for trade,

In [37]:
data = {'Predicted_Labels': predicted_labels}
df = pd.DataFrame(data)
df.to_csv('data/predicted_labels.csv', index=False)